In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import NMF
from  xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

prop_16=pd.read_csv('../input/properties_2016.csv')#,dtype={'garagetotalsqft':np.object,'pooltypeid7':np.object,\
        #'propertylandusetypeid':np.object,'numberofstories':np.object,'taxamount':np.object})
train_16=pd.read_csv('../input/train_2016_v2.csv')
#z_data=pd.read_excel('../input/zillow_data_dictionary.xlsx')
submission=pd.read_csv('../input/sample_submission.csv')
submission=submission.rename(columns={'ParcelId': 'parcelid'}) 



In [ ]:
print(prop_16.columns.values)
#print(prop_16.dtypes)
print('=======================')
print(train_16.columns.values)
print('=======================')
print(prop_16.head(10))
#print(prop_16.dtypes)
print('=======================')
print(train_16.head(10))




The properties data file contains alot of null values, we will need to analyse and impute them. So let's try to analyse the training data first

In [ ]:
#train_16['logerror'].hist(bins=400)
sns.distplot(train_16['logerror'])

In [ ]:
sns.regplot('parcelid','logerror',data=train_16)

In [ ]:
sns.stripplot(x='transactiondate',y='logerror',data=train_16)

In [ ]:
plt.figure(figsize=(10,40))
sns.violinplot(y='transactiondate',x='logerror',data=train_16)

In [ ]:
print(prop_16.shape)
prop_null=prop_16.isnull().sum()

prop_null.plot(kind='bar',figsize=(20,10),color='grey')
plt.xticks(fontsize=20)
plt.xlabel('Feature Name')
plt.yticks(fontsize=20)
plt.ylabel('Null Value Count')
#sns.distplot(prop_16.isnull().sum())

Some of the features contain alot of  null values.  Calculating the percentage of missing values and dropping those that are>80%

In [ ]:
print(prop_16.shape)
prop_null=prop_16.isnull().sum()/prop_16.shape[0]*100

prop_null.plot(kind='bar',figsize=(20,10),color='grey')
plt.xticks(fontsize=20)
plt.xlabel('Feature Name')
plt.yticks(fontsize=20)
plt.ylabel('Null Value Count')

In [ ]:

imp_feature=prop_16.loc[:,prop_null<80]#imp_feature=prop_16.loc[:,prop_16[prop_null<80]]
imp_feature.fillna(imp_feature.median(),inplace=True)

In [ ]:
#imp_feature['propertycountylandusecode'].fillna(imp_feature['propertycountylandusecode'].value_counts().index[0],inplace=True)
#imp_feature['propertyzoningdesc'].fillna(imp_feature['propertyzoningdesc'].value_counts().index[0],inplace=True)
print(imp_feature.isnull().any())
#print(imp_feature['propertycountylandusecode'].head(3))
#print(imp_feature['propertyzoningdesc'].head(3))


In [ ]:
train_16 = pd.merge(train_16,imp_feature, on='parcelid', how='left')
sub_x=pd.merge(submission,imp_feature, on='parcelid', how='left')

In [ ]:
#y=train_16['logerror']
#train_16=pd.concat(train_16,pd.get_dummies(train_16['propertycountylandusecode']),axis=1)
#train_16.drop('logerror',axis=1,inplace=True)
#train_16.drop('parcelid',axis=1,inplace=True)


In [ ]:
def transac(df):
    df['transactiondate']=pd.to_datetime(df['transactiondate'])
    df['transaction_month'] = df['transactiondate'].dt.month
    df['transaction_year'] = df['transactiondate'].dt.year
#train_16['transactiondate']=pd.to_datetime(train_16['transactiondate'])
#train_16['transaction_month'] = train_16['transactiondate'].dt.month
#train_16['transaction_year'] = train_16['transactiondate'].dt.year

transac(train_16)
#transac(sub_x)
print(train_16.dtypes)
print(sub_x.dtypes)
#XGB = XGBRegressor()
#XGB.fit(X_train, X_test)

In [ ]:
#train_16=pd.get_dummies(train_16,columns=['propertyzoningdesc','propertycountylandusecode'])
#print(train_16.dtypes)

In [ ]:
print(train_16.columns.values)
print(train_16.shape)

In [ ]:
y=train_16['logerror'].values.astype(np.float32)

In [ ]:
#train_16.drop(['logerror','parcelid','transactiondate'],axis=1,inplace=True)

train_16.drop(['logerror','parcelid','transactiondate','propertyzoningdesc','propertycountylandusecode'],axis=1,inplace=True)
sub_x.drop(['propertyzoningdesc','propertycountylandusecode'],axis=1,inplace=True)
sub_x.drop(['201610','201611','201612','201710','201711','201712'],axis=1,inplace=True)
sub_x.drop('parcelid',axis=1,inplace=True)
train_16.drop(['transaction_month', 'transaction_year'],axis=1,inplace=True)

In [ ]:
print(train_16.columns.values)
print(sub_x.columns.values)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_16, y, random_state=0)
#rf = RandomForestRegressor(max_depth=150,random_state=42) score=0.08, n_estimators=30 s=0.076
rf = RandomForestRegressor(n_estimators=30,max_depth=100,random_state=42,max_features=5)#,criterion='mae')
rf.fit(X_train, Y_train)
#print((rf.feature_importances_))

In [ ]:
#important_features = pd.Series(data=rf.feature_importances_,index=X_train.columns.values)
#important_features.sort_values(ascending=False,inplace=True)
#important_features.plot(figsize=(10,400),kind='barh')


In [ ]:
y_pred=rf.predict(X_test)

In [ ]:
df_y=abs(Y_test-y_pred)
MAE=df_y.sum()/len(Y_test)#.shape[0]
print(MAE)

In [ ]:
y_sub=rf.predict(sub_x)
sample_file = pd.read_csv('../input/sample_submission.csv') 
for c in sample_file.columns[sample_file.columns != 'ParcelId']:
    sample_file[c] = y_sub

sample_file.to_csv('randomforest_resultv0.2.csv', index=False, float_format='%.4f')

#MAE <- sum(abs(y-y_pred)) / length(y)
#print(rf.score(X_train,Y_train))
#print(rf.score(X_test,Y_test))

In [ ]:
#from sklearn.feature_selection import RFE
#select = RFE(RandomForestRegressor(max_depth=200,random_state=42),n_features_to_select=20)
#select.fit(X_train,Y_train)
#X_train_rfe=select.transform(X_train)
#X_test_rfe=select.transform(X_test)


In [ ]:
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix
#y_pred=rf.predict(X_test)

#print(rf.score(X_train,Y_train))
#print(rf.score(X_test,Y_test))
#print(y_pred)

      